In [2]:
import pandas as pd
import io
import numpy as np
import pymc3 as pm

from sklearn.preprocessing import StandardScaler


ImportError: cannot import name 'asscalar' from 'numpy' (/home/mathis/miniconda3/envs/asthme/lib/python3.10/site-packages/numpy/__init__.py)

In [2]:
with open('german.data', 'r') as f:
    data_str = f.read()
f.close()

In [3]:
# reading file 
data = io.StringIO(data_str)
df = pd.read_csv(data,sep=' ', header=None)
df.columns = ['status', 'months', 'credit', 'purpose', 'amount', 'savings', 'employment', 'rate', 'sex', 'other', 'residence',
            'property', 'age', 'plan', 'housing', 'nb_credits', 'job', 'liability', 'phone', 'foreign', 'target']
df.loc[df['target']==1, 'target'] =-1
df.loc[df['target']==2, 'target'] =1

# replace categories by integers
std_scaler = StandardScaler()

for c in df.columns:
    if df[c].dtype != np.int64:
        # convert to categorical
        df[c] = df[c].astype('category').cat.codes
    else:
        # normalize
        df[c] = (df[c] -df[c].mean())/df[c].std()

df.head()

NameError: name 'StandardScaler' is not defined

In [5]:
df.iloc[:,:-1].shape

(1000, 20)

In [6]:
df.columns

Index(['status', 'months', 'credit', 'purpose', 'amount', 'savings',
       'employment', 'rate', 'sex', 'other', 'residence', 'property', 'age',
       'plan', 'housing', 'nb_credits', 'job', 'liability', 'phone', 'foreign',
       'target'],
      dtype='object')

In [7]:
model = 'target ~ status + months + credit + purpose + amount + savings + employment + rate + sex \
    + other + residence + property + age + plan + housing + nb_credits + job + liability + phone \
        + foreign'

In [8]:
with pm.Model() as logistic_model:
    pm.glm.GLM.from_formula(formula=model, data=df, family=pm.glm.families.Binomial())

In [9]:
logistic_model.basic_RVs

[Intercept ~ Flat,
 status ~ Normal,
 months ~ Normal,
 credit ~ Normal,
 purpose ~ Normal,
 amount ~ Normal,
 savings ~ Normal,
 employment ~ Normal,
 rate ~ Normal,
 sex ~ Normal,
 other ~ Normal,
 residence ~ Normal,
 property ~ Normal,
 age ~ Normal,
 plan ~ Normal,
 housing ~ Normal,
 nb_credits ~ Normal,
 job ~ Normal,
 liability ~ Normal,
 phone ~ Normal,
 foreign ~ Normal,
 y ~ Binomial]

In [10]:
with logistic_model:
    trace = pm.sample(200)

/tmp/ipykernel_12459/558954564.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(200)
Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [foreign, phone, liability, job, nb_credits, housing, plan, age, property, residence, other, sex, rate, employment, savings, amount, purpose, credit, months, status, Intercept]


In [ ]:
lower=-10**6
higher=10**6
with pm.Model() as model:
    # both alpha and beta are given weak zero-mean normal prior with variance =100
    alpha = pm.Normal('a', mu=0, sigma=np.sqrt(100))
    beta = pm.Normal('b',mu =0, sigma=np.sqrt(100), shape=(20))
    #beta_0=pm.Uniform('beta_0', lower=lower, upper= higher)
    #beta_age=pm.Uniform('beta_age', lower, higher)
    # define proba of belonging to class 1 
    p = pm.Deterministic('p', pm.math.sigmoid(alpha+beta@df.iloc[:,:-1].T))

with model:
    #fit the data 
    observed=pm.Bernoulli("target", p, observed=df['target'])
    start= pm.find_MAP()
    step= pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(2500, step=step, start=start)
    burned_trace=trace[15000:]
                      


/tmp/ipykernel_4805/211686632.py:19: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace=pm.sample(2500, step=step, start=start)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [b]
>Metropolis: [a]
